In [1]:
import os
import glob
import rawpy
import exifread
import cv2 as cv
import numpy as np
from GaussianPyramid import HDRPyramid
import matplotlib.pyplot as plt

In [2]:
def generate_tiles(image, tile_size, stride):
    if isinstance(tile_size, int):
        tile_size = (tile_size, tile_size)
    if isinstance(stride, int):
        stride = (stride, stride)
    tiles = []
    h, w = image.shape
    tile_size_h, tile_size_w = tile_size
    stride_h, stride_w = stride
    for i in range(0, h - tile_size_h + 1, stride_h):
        row = []
        for j in range(0, w - tile_size_w + 1, stride_w):
            tile = image[i : i + tile_size_h, j : j + tile_size_w]
            row.append(tile)
        tiles.append(row)
    tiles = np.array(tiles)
    return tiles

def isTypeInt(image):
    return image.dtype in [np.uint8, np.uint16, np.uint32, np.uint64, np.int8, np.int16, np.int32, np.int64, np.uint]

def downsample_bayer(image):
    R = image[0 :: 2, 0 ::2] # 红色通道，取偶数行和偶数列的像素
    G1 = image[0 :: 2, 1 :: 2] # 绿色通道1， 取偶数行和奇数列的像素
    G2 = image[1 :: 2, 0 :: 2] # 绿色通道2，取奇数行和偶数列的像素
    B = image[1 :: 2,  1 :: 2] # 蓝色通道，取奇数行和奇数列的像素
    if isTypeInt(image):
        return np.right_shift(R + G1 + G2 + B + 2, 2)
    else:
        return (R + G1 + G2 + B) * 0.25

In [3]:
refIndex = 0
rawBayers = []
burstPath = "/home/lianghao/Documents/Learning/ISP/Image/IPOL_hdrplus_gopro_raw_bursts/hand_iso1600/"
alignmen_params = {
				'mode': 'bayer',  # images are single image Bayer / Color Filter Arrays
				'tuning': {
					# WARNING: these parameters are defined fine-to-coarse!
					'factors': [1, 2, 4, 4],
					'tileSizes': [16, 16, 16, 8],
					'searchRadia': [1, 4, 4, 4],
					'distances': ['L1', 'L2', 'L2', 'L2'],
					'subpixels': [False, True, True, True]  # if you want to compute subpixel tile alignment at each pyramid level
				},
				# rawpy parameters for images with motion fields
				'rawpyArgs': {
					'demosaic_algorithm' : rawpy.DemosaicAlgorithm.AHD,  # used in HDR+ supplement
					'half_size' : False,
					'use_camera_wb' : True,
					'use_auto_wb' : False,
					'no_auto_bright': True,
					'output_color' : rawpy.ColorSpace.sRGB,  # sRGB
					'output_bps' : 8
				},
				'writeMotionFields': False,
				'writeAlignedTiles': False
			}

rawPathList = glob.glob(os.path.join(burstPath, "*.dng"))
rawPathList.sort()
for rawPath in rawPathList:
    with rawpy.imread(rawPath) as rawObject:
        rawBayers.append(rawObject.raw_image.copy())

In [4]:
with rawpy.imread(rawPathList[refIndex]) as refRawpy:
    blackLevel = refRawpy.black_level_per_channel.copy()
    whiteLevel = refRawpy.white_level

In [5]:
h, w = rawBayers[refIndex].shape
if alignmen_params["mode"] == "bayer":
    tileSize = 2 * alignmen_params['tuning']['tileSizes'][0]
else:
    tileSize = alignmen_params['tuning']['tileSizes'][0]

In [6]:
# padding setting 
padding_height = (tileSize - h % tileSize) % tileSize
padding_width = (tileSize - w % tileSize) % tileSize
padding_overlap_height, padding_overlap_width = tileSize // 2, tileSize // 2
padding_top = padding_overlap_height
padding_bottom = padding_overlap_height + padding_height
padding_left = padding_overlap_width
padding_right = padding_overlap_width + padding_left

images_padding = []
for i in range(len(rawBayers)):
    im = np.pad(rawBayers[i], ((padding_top, padding_bottom), (padding_left, padding_right)), mode="symmetric")
    images_padding.append(im)

reference = images_padding[refIndex]
alternatives = [images_padding[i] for i in range(len(images_padding)) if i != refIndex]

implements the coarse-to-fine alignment on 4-level gaussian pyramids as defined in Algorithm 1 of Section 3 of the IPOL article.

In [7]:
# factors, tileSizes, distances, searchRadia and subpixels are described fine-to-coarse
factors = [1, 2, 4, 4]
tile_sizes =  [16, 16, 16, 8]
search_radia = [1, 4, 4, 4]
distances = ['L1', 'L2', 'L2', 'L2']
subpixels = [False, True, True, True]

upsampling_factors = factors[1 : ] + [1]
previous_tile_sizes = tile_sizes[1:] + [None]

# If dealing with raw images, 2x2 bayer pixels block are averaged
# (motion can then only be multiples of 2 pixels in original image size)
# 对参考帧进行下采样2倍
imRef = downsample_bayer(reference)
tile_size = 2 * tile_sizes[0]
refTiles = generate_tiles(reference, tile_size, tile_size // 2)

In [8]:
# 存储对齐瓦片
aligned_tiles = np.empty(((len(images_padding),) + refTiles.shape), dtype=refTiles.dtype) 
aligned_tiles[0] = refTiles # 参考帧的瓦片作为对齐的初始值
# 备选帧的瓦片对参考帧的瓦片的移动向量
motion_vectors = np.empty((len(alternatives), refTiles.shape[0], refTiles.shape[1], 2), dtype=int)
# 构建参考帧的4层从粗corse到细fine的金字塔
hdrPyramid = HDRPyramid()
refPyramid = hdrPyramid.gaussian_pyramid(image=imRef, factors=factors, ksize=5)

In [9]:
# 将每个备选帧对齐到参考帧
for i, alternateImg in enumerate(alternatives):
    # downsample bayer raw to grayscale
    im_alter = downsample_bayer(alternateImg)
    # 生成备选帧的4层由粗到细的金字塔
    alternatePyramid = hdrPyramid.gaussian_pyramid(im_alter, factors)
    # 金字塔由粗到细渐进式地对齐
    aligments = None
    for lv in range(len(refPyramid)):
        pass

金字塔对齐

In [ ]:
def align_pyramid(refer_pyramid, alter_pyramid, up_factor, 
                  tile_size, pre_tile_size, search_radia, measure="L1", subpixel=True, alignments=None):
    pass